# The mongod

# Mongod Options

In [ ]:
mongod --help

### dbpath

In [ ]:
mongod --dbpath <directory path>

### port

In [ ]:
mongod --port <port number>

### auth

In [ ]:
mongod --auth

### bind_ip

In [ ]:
mongod --bind_ip 123.123.123.123

In [ ]:
mongod --bind_ip localhost,123.123.123.123

### 練習01

In [ ]:
mongod --dbpath "/home/spark/mongodb/data" --port 27000

# Configuration File

See MongoDB documentation for more information about [command line options](https://www.mongodb.com/docs/manual/reference/program/mongod/#options) and [configuration file options](https://www.mongodb.com/docs/manual/reference/configuration-options/).

In [ ]:
mongod

In [ ]:
mongod --dbpath /data/db --logpath /data/log/mongod.log

In [ ]:
mongod --dbpath /data/db --logpath /data/log/mongod.log --fork

In [ ]:
mongod --dbpath /data/db --logpath /data/log/mongod.log --fork --auth --port 27017 --replSet "M103" --keyFile /data/keyfile --bind_ip "127.0.0.1,192.168.103.100" --tlsMode requireTLS --tlsCAFile "/etc/tls/TLSCA.pem" --tlsCertificateKeyFile "/etc/tls/tls.pem"

### 啟動

In [ ]:
mongod --config "/etc/mongod.conf"

In [ ]:
mongod -f "/etc/mongod.conf"

### 練習01

In [ ]:
storage:
  dbPath: "/home/spark/mongodb/data/"
systemLog:
  path: "/home/spark/mongodb/log/mongo.log"
  destination: "file"
net:
  port: 27000
security:
  authorization: "enabled"

# File Structure

In [ ]:
ls -l /data/db

In [ ]:
ls -l /data/db/diagnostic.data

In [ ]:
ls -l /data/db/journal

In [ ]:
ls /tmp/mongodb-27017.sock

# Basic Commands

In [ ]:
db.createUser()
db.dropUser()

In [ ]:
db.<collection>.renameCollection()
db.<collection>.createIndex()
db.<collection>.drop()

### 範例01

In [ ]:
# 把 rrecord 改成 record
db.rrecord.renameCollection("record")

---

In [ ]:
db.dropDatabase()
db.createCollection()

### 範例02

In [ ]:
# 建立 demoCollection ，這個指令有很多選項可選
db.createCollection('demoCollection')

---

In [ ]:
db.serverStatus()

In [ ]:
db.runCommand({
   "createIndexes":"<collection_name>",
       "indexes":[
          {
             "key":{ "product": 1 },
             "name": "name_index"
          }
       ]
    }
 )

In [ ]:
db.<collection>.createIndex(
  { "product": 1 },
  { "name": "name_index" }
)

In [ ]:
db.<collection>.createIndex

# Logging Basics

In [ ]:
db.getLogComponents()

In [ ]:
db.setLogLevel(0, "index")

---

In [ ]:
db.adminCommand({ "getLog": "global" })

In [ ]:
tail -f /data/db/mongod.log

---

In [ ]:
db.products.update( { "sku" : 6902667 }, { $set : { "salePrice" : 39.99} } )

In [ ]:
# 看不到修改，要去看 profiler，或者調 level
grep -i 'update' /data/db/mongod.log

---

# Profiling the Database

### 範例01

In [ ]:
# 新建一個資料庫
use newDB

# db.getProfilingLevel() 已經 Deprecated 
db.getProfilingStatus()

db.setProfilingLevel(1)

db.getProfilingStatus()

# mongoDB 會建立一個 collection 稱作 system.profile
show collections

db.setProfilingLevel(1, { slowms: 0 })

db.getProfilingStatus()

db.new_collection.insertOne({ a: 1 })

db.system.profile.find().pretty()

db.new_collection.find({ a: 1 })

db.system.profile.find().pretty()

---

In [ ]:
# show collections 的效果
db.runCommand({listCollections: 1})

In [ ]:
db.getCollectionNames()  # 跟 show collections 有相同的效果

---

### 範例02

In [ ]:
Update your configuration file such that:

# mongod sends logs to /home/spak/mongo/log/mongo.log
# mongod is forked and run as a daemon (this will not work without specifying logpath)(重要)

In [ ]:
storage:
  dbPath: "/home/spark/mongodb/data/"
systemLog:
  path: "/home/spark/mongodb/log/mongo.log"
  destination: "file"
net:
  bindIp: "127.0.0.1"
  port: 27000
#security:
#  authorization: "enabled"
processManagement:
  fork: true

# Basic Security

### Authentication Mechanisms (client)

### Authentication Mechanisms (cluster)

### Authorization: Role Based Access Control

### 範例01

In [ ]:
# 從 security 內開啟 authorization: true
cat /etc/mongod.conf

In [ ]:
mongo --host 127.0.0.1:27017
    
use admin 

# 授權被大大的限制
db.stats()

In [ ]:
db.createUser(
    {
        user: "root",
        pwd: "root",
        roles: [ "root" ]
    }
)

In [ ]:
# The localhost exception at this point in time is exhausted
db.createUser(
    {
        user: "JDoe",
        pwd: "JDoe123",
        roles: [ "readWrite" ]
    }
)

In [ ]:
# 也可以在 admin 資料庫用 db.auth("root", "root") 方法
mongo --host 127.0.0.1:27017 --username root --password root --authenticationDatabase admin

In [ ]:
use admin

# 現在驗證成功
db.stats()

---

# Built-In Roles

### Roles in MongoDB

### Privilege

In [ ]:
- Resource
- Actions allowed over a resource   

<img src="2.png">

<img src="1.png">

### Built-in roles

<img src="3.png">

<img src="4.png">

### 範例01

In [ ]:
# 在 linux 建一個帳號有 root 的 role
mongo admin -u root -p root123

In [ ]:
# 這個腳色沒辦法操控資料
db.createUser({
    user: "security_officer",
    pwd: "h3110th3r3",
    roles: [ { db: "admin", role: "userAdmin" } ]
})

<img src="5.PNG">

In [ ]:
# 只能操作跟 db 相關的指令
db.createUser({
    user: "dba",
    pwd: "cllynd3rs",
    roles: [ { db: "m103", role: "dbAdmin" } ]
})

<img src="6.PNG">

In [ ]:
# 再增加 roles 給之前創的帳號 dba
db.grantRolesToUser( "dba", [ { db: "playground", role: "dbOwner" } ])

In [ ]:
dbOwner

- The database owner can perform any administrative action on the database.
- The role combines the privileges granted by the readWrite, dbAdmin, userAdmin roles.

In [ ]:
db.runCommand({
    rolesInfo: { role: "dbOwner", db: "demo"},
    showPrivileges: true
})

# Server Tools Overview

In [ ]:
find /usr/bin/ -name "mongo*"

In [ ]:
mkdir -p ~/first_mongod
mongod --port 30000 --dbpath ~/first_mongod --logpath ~/first_mongod/mongodb.log --fork

In [ ]:
mongostat --help
mongostat --port 30000

---

### mongodump 與 mongorestore 是 BSON 相關

In [ ]:
mongodump --help
mongodump --port 30000 -u "m103-admin" -p "m103-pass" --authenticationDatabase "admin" --db exampleDB --collection students
ls
cd dump/
ls
cd exampleDB
ls
cat students.metadata.json

In [ ]:
# 不用指定 -d -c 因為資訊都被存在 dump 裡面的 metadata.json
mongorestore --drop --port 30000 -u "m103-admin" -p "m103-pass" --authenticationDatabase "admin" dump/

### mongoimport 與 mongoexport 是 JSON 相關

In [ ]:
mongoexport --help
# 沒有指定 output，會在螢幕顯示。
mongoexport --port 30000 -u "m103-admin" -p  "m103-pass" --authenticationDatabase "admin" --db exampleDB --collection students 

In [ ]:
# 有指定 output
mongoexport --port 30000 -u "m103-admin" -p  "m103-pass" --authenticationDatabase "admin" --db exampleDB --collection students -o students.json
tail students.json

In [ ]:
tail students.json

In [ ]:
# 如果沒有指定 Database 跟 Collection，會被放到 test 資料庫，用檔案名當 collection 的名稱。
# 格式如果最外面是 Array 要加個選項 --jsonArray
mongoimport --drop  --port 30000 -u "m103-admin" -p "m103-pass" --authenticationDatabase "admin" students.json